# Study on the Effects of the D614G Mutation on the Infectivity of SARS-CoV-2 Using California as a Case Study

by Joshua Delgadillo, Lauren Craft, Adam Perlin, Efrain Rangel, Brett Triebold, Ritta Liu

 # Background

Our team chose to research the novel Coronavirus. In 2020 SARS-CoV-2 began spreading across the world and there has been an widespread effort to understand the virus and research ways to combat it. Bioinformatics takes the forefront in understanding the genetic makeup of the virus and its mutations. Our contributions to the research into understanding SARS-CoV-2 include tracing similarities between SARS-CoV-2 and two viruses that have previously spread. These two viruses are SARS-CoV-1 and MERS-CoV. Much of our research and data collection involved taking sequences of these three viruses and examining the relationships between them in order to extract data to determine similarities and differences between them.

The data used in our research came from the National Center for Biotechnology Information (NCBI). The NCBI contains a vast amount of bioinformatics related databases including several databases which contain virus specific DNA sequences. The database that we focused our data extraction on was the NCBI Nucleotide Database. From this database we were able to collect several sequence samples to use in our research.

# Project Overview

After successfully collecting sequence data for SARS-CoV-1, SARS-CoV-2 and MERS-CoV, we analyzed the similarities and differences between example sequences of each of the viruses. We wanted to expand upon this research and begin looking into mutations of SARS-CoV-2 that have appeared since the initial identification. The specific mutation that we focused on is the D614G mutation.

The D614G mutation is an A-to-G missense mutation at position 23,403 that causes the substitution of aspartic acid with glycine (1). This causes a change in the structure of the viral spike glycoprotein. Virus with this mutation has since been named clade G and was first detected on January 21, 2020 (2). Due to the increasing global dominance of clade G, it is suspected that it increases susceptibility. A study found retroviruses with SG614 infected ACE2-expressing cells marked more efficiently than those with SD614 (6). It was initially mainly prevalent in European regions (1). In Italy there was an outbreak during the early stage of the global pandemic, resulting in 400 deaths in one month (3). 87% of samples taken from Italy were found to have the D614G mutation.

California will be used as a case study. Samples will be taken from patients with Covid-19 all over California, and the prevalence of the D614G mutation among the California population can be deduced. We have randomly selected 30 samples from the NCBI, and searched for mutations on the spike glycoprotein (S protein) (5). Since the mutation has been more commonly found in Europe, it would likely have been introduced to the California population later on. The presence of the D614G mutation among Californian cases could infer increased infectivity.


# Implementation

### Smith-Waterman Algorithm

Our implementation began with the comparison between SARS-CoV-1, SARS-CoV-2 and MERS-CoV. In order to perform these comparisons we utilized the Smith-Waterman Algorithm. Earlier research performed to analyze the similarities and differences between the viruses utilized the Needleman-Wunsch Algorithm which finds the optimal global alignment of sequences. However, our decision to use Smith-Waterman came from its impressive ability to find the best local alignment. We use the individual local alignment scores to draw conclusions about overall viral genome similarity; i.e., if all local alignments exhibit high similarity and local alignments take up a large proportion of the genome then the two genomes are likely to be quite similar.


Pseduo code:

~~~

Given: String s1 with length m , String s2 with length n

    // initialize matrix, M
    
    // score cells in matrix
    for i=1 to m
        for j=1 to n
        
            // initialization: max is 0
            max = 0 
            
            // first comparison: west cell (deletion)
            score = M[i][j-1] + gapScore
            if( score > max )
                max = score
            
            // second comparison: north cell (insertion)
            score = M[i-1][j] + gapScore
            if( score > max )
                max = score
            
            // last comparison: north-west cell (alignment)
            base1 = s1[j-1]
            base2 = s2[i-1]
            
            if( base1 == base2 )              // match
                alignmentScore = matchScore
            else                              // mismatch
                alignmentScore = mismatchScore
            
            score = M[i-1][j-1] + alignmentScore
            if( score > max )
                max = score
            
            // finished all comparisons
            M[i][j] = max
    
    // return completed matrix
    return M
~~~

### BLAST

We chose to utilize "The Basic Local Alignment Search Tool (BLAST), in order to perform our sequence analysis. 

From the [BLAST website]: BLAST finds regions of local similarity between sequences. The program compares nucleotide or protein sequences to sequence databases and calculates the statistical significance of matches."

In our research, we found out BLAST actually uses a faster (but less precise) version of Smith-Waterman. A heuristic is used to eliminate sequences that are not likely to be a good local alignment. The shortcut was necessary because the algorithm is too slow when ran with large datasets. source: [The NCBI Handbook]

[BLAST website]: https://blast.ncbi.nlm.nih.gov/Blast.cgi
[The NCBI Handbook]: https://www.unmc.edu/bsbc/docs/NCBI_blast.pdf

A function to print the contents of .fasta files:

In [53]:
from Bio import SeqIO
import pandas as pd 
  
def print_fasta(virus_name, file_name):
    # initialize list of lists 
    data = []

    fasta_sequences = SeqIO.parse(open(file_name),'fasta')
    with open(file_name) as out_file:
        for fasta in fasta_sequences:
            name, sequence = fasta.id, str(fasta.seq)
            data.append([name, sequence])

    # Create the pandas DataFrame 
    df = pd.DataFrame(data, columns = [virus_name, 'sequence']) 

    # print dataframe. 
    print(df) 
        

Printing one example sequence of each of the viruses:

In [54]:
from glob import glob

data_dir = "test_data"
sample_types = [
    "SARS-CoV-1",
    "SARS-CoV-2",
    "MERS-CoV"
]

for sample_type in sample_types:
    sample = glob(data_dir+"/{}*.fasta".format(sample_type))
    print_fasta(sample_type, sample[0])
    print()

   SARS-CoV-1                                           sequence
0  ATO98191.1  MESLVLGVNEKTHVQLSLPVLQVRDVLVRGFGDSVEEALSEAREHL...

   SARS-CoV-2                                           sequence
0  QOP81761.1  MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...

     MERS-CoV                                           sequence
0  AVV62535.1  MSSVAGVVAQGARNRYRAALNNEKRPDHVSLTVPCCGTGDLAEHLS...



#### Database Creation for Utilizing BLAST

In [55]:
%%bash
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in test_data/SARS-CoV-1-ATO98191.fasta -dbtype prot
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in test_data/SARS-CoV-2-QOP81761.fasta -dbtype prot
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in test_data/MERS-CoV-AUM60013.fasta -dbtype prot



Building a new DB, current time: 11/30/2020 22:29:41
New DB name:   /home/jupyter-jdelga26/448/test_data/SARS-CoV-1-ATO98191.fasta
New DB title:  test_data/SARS-CoV-1-ATO98191.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000217199 seconds.




Building a new DB, current time: 11/30/2020 22:29:41
New DB name:   /home/jupyter-jdelga26/448/test_data/SARS-CoV-2-QOP81761.fasta
New DB title:  test_data/SARS-CoV-2-QOP81761.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000206947 seconds.




Building a new DB, current time: 11/30/2020 22:29:41
New DB name:   /home/jupyter-jdelga26/448/test_data/MERS-CoV-AUM60013.fasta
New DB title:  test_data/MERS-CoV-AUM60013.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000194073 seconds.




Running blast on each pair:

In [56]:
%%bash
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./test_data/SARS-CoV-1-ATO98191.fasta -db ./test_data/SARS-CoV-2-QOP81761.fasta -evalue 1e-6 -num_threads 16 -out ./test_data/blast_S1_S2.txt
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./test_data/SARS-CoV-1-ATO98191.fasta -db ./test_data/MERS-CoV-AUM60013.fasta -evalue 1e-6 -num_threads 16 -out ./test_data/blast_S1_M.txt
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./test_data/SARS-CoV-2-QOP81761.fasta -db ./test_data/MERS-CoV-AUM60013.fasta -evalue 1e-6 -num_threads 16 -out ./test_data/blast_S2_M.txt

BLAST results of SARS-CoV-1 and SARS-CoV-2:

In [57]:
!grep -A1 "Score = " ./test_data/blast_S1_S2.txt | head -2

 Score = 12938 bits (33578),  Expect = 0.0, Method: Compositional matrix adjust.
 Identities = 6126/7101 (86%), Positives = 6599/7101 (93%), Gaps = 33/7101 (0%)


BLAST results of SARS-CoV-1 and MERS-CoV:

In [58]:
!grep -A1 "Score = " ./test_data/blast_S1_M.txt | head -2

 Score = 6032 bits (15650),  Expect = 0.0, Method: Compositional matrix adjust.
 Identities = 3061/6037 (51%), Positives = 4050/6037 (67%), Gaps = 195/6037 (3%)


BLAST results of SARS-CoV-2 and MERS-CoV:

In [59]:
!grep -A1 "Score = " ./test_data/blast_S2_M.txt | head -2

 Score = 6061 bits (15723),  Expect = 0.0, Method: Compositional matrix adjust.
 Identities = 3055/5946 (51%), Positives = 4030/5946 (68%), Gaps = 177/5946 (3%)


Commands to clean up the test_data directory:

In [60]:
!rm ./test_data/*.fasta.*
!rm ./test_data/*.txt

#### BLAST Database Creation

In [61]:
%%bash
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in data/SARS-CoV-1-Sequences.fasta -dbtype prot
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in data/SARS-CoV-2-Sequences.fasta -dbtype prot
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in data/MERS-CoV-Sequences.fasta -dbtype prot



Building a new DB, current time: 11/30/2020 22:29:41
New DB name:   /home/jupyter-jdelga26/448/data/SARS-CoV-1-Sequences.fasta
New DB title:  data/SARS-CoV-1-Sequences.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 157 sequences in 0.00471306 seconds.




Building a new DB, current time: 11/30/2020 22:29:41
New DB name:   /home/jupyter-jdelga26/448/data/SARS-CoV-2-Sequences.fasta
New DB title:  data/SARS-CoV-2-Sequences.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 108 sequences in 0.00388503 seconds.




Building a new DB, current time: 11/30/2020 22:29:41
New DB name:   /home/jupyter-jdelga26/448/data/MERS-CoV-Sequences.fasta
New DB title:  data/MERS-CoV-Sequences.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 135 sequences in 0.00465202 seconds.




Run blast on every pair

In [62]:
%%bash
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./data/SARS-CoV-1-Sequences.fasta -db ./data/SARS-CoV-2-Sequences.fasta -evalue 1e-6 -num_threads 16 -out ./data/blast_S1_S2.txt
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./data/SARS-CoV-1-Sequences.fasta -db ./data/MERS-CoV-Sequences.fasta -evalue 1e-6 -num_threads 16 -out ./data/blast_S1_M.txt
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./data/SARS-CoV-2-Sequences.fasta -db ./data/MERS-CoV-Sequences.fasta -evalue 1e-6 -num_threads 16 -out ./data/blast_S2_M.txt

Print out results of comparisons (top alignment score in every output file)

In [63]:
%%bash
echo "SARS-CoV-1 x SARS-CoV-2"
grep -A1 -m1 "Score =" ./data/blast_S1_S2.txt | head -2
echo "SARS-CoV-1 x MERS-CoV"
grep -A1 -m1 "Score =" ./data/blast_S1_M.txt | head -2
echo "SARS-CoV-2 x MERS-CoV"
grep -A1 -m1 "Score =" ./data/blast_S2_M.txt | head -2

SARS-CoV-1 x SARS-CoV-2
 Score = 12927 bits (33549),  Expect = 0.0, Method: Compositional matrix adjust.
 Identities = 6120/7103 (86%), Positives = 6595/7103 (93%), Gaps = 37/7103 (1%)
SARS-CoV-1 x MERS-CoV
 Score = 5983 bits (15522),  Expect = 0.0, Method: Compositional matrix adjust.
 Identities = 3066/6175 (50%), Positives = 4083/6175 (66%), Gaps = 262/6175 (4%)
SARS-CoV-2 x MERS-CoV
 Score = 5995 bits (15553),  Expect = 0.0, Method: Compositional matrix adjust.
 Identities = 3041/5978 (51%), Positives = 4002/5978 (67%), Gaps = 202/5978 (3%)


In [65]:
import re
import numpy as np

def average_identity_rate_for_all_alignments(blast_output_file):
    f = open(blast_output_file)
    total_positive_frac = 0
    n_significant_scores = 0

    for line in f.readlines():
        m = re.match("\W*Identities = (\d+)\/(\d+)", line)
        if m is not None:
            num = float(m.group(1))
            denom = float(m.group(2))
            total_positive_frac += num/denom
            n_significant_scores += 1
            
    return total_positive_frac / n_significant_scores

pairings = [('S1', 'S2'), ('S1', 'M'), ('S2', 'M')]
rows = np.zeros((3 ,3))
df = pd.DataFrame(rows, columns=['S1', 'S2', 'M'], index=['S1', 'S2', 'M'])
df.loc['S1', 'S1'] = df.loc['S2', 'S2'] = df.loc['M', 'M'] = 100
for pair in pairings:
    f = './data/blast_' + '_'.join(pair) + '.txt'
    df.loc[pair[0], pair[1]] = df.loc[pair[1], pair[0]] = round(average_identity_rate_for_all_alignments(f)*100)

df

,S1,S2,M
S1,100.0,84.0,40.0
S2,84.0,100.0,38.0
M,40.0,38.0,100.0


Clean up data directory

In [66]:
!rm ./data/*.fasta.*

### Mutation Research

After analyzing the similarities and differences between the SARS-CoV-1, SARS-CoV-2 and MERS-CoV sequences, we began futhering our research of SARS-CoV-2 by inspecting the D614G mutation. The D614G mutation causes a change in the structure of the viral spike glycoprotein.

In order to focus the study, we utilized California as our testing sample and focus to act as a case study of the D614G missense mutation. We analyzed 30 samples taken from the NCBI database and searched to see if they contained the mutation.

The functions below read in the sequence data and find which contain the mutation.

In [67]:
def get_sequence(filename):
    file = open("./mutation_data/" +filename)
    sequence = ""
    count = 0
    
    for line in file:
        if count != 0:
            sequence += line.strip()
        count += 1
        
    return sequence
        
    
def find_mutations():
    out_file_1 = open("./mutation_data/results.txt", "w")
    out_file_2 = open("./mutation_data/differences.txt", "w")
    reference_genome = get_sequence("reference_seq.fasta")
    mutation_filename = "seq_00.fasta"
    
    mutations = []
    
    for i in range(1, 31):
        temp = mutation_filename[:4] + f'{i:02}' + mutation_filename[6:]
        mutation = get_sequence(temp)
        mutations.append(mutation)
    
    for i in range(len(mutations)):
        mutation = mutations[i]
        
        if mutation[23402] == 'G':
            out_file_1.write(str(i + 1) + " - " + mutation[23402] + "\n")
    
    for i in range(len(mutations)):
        out_file_2.write(str(i) + " - differences")
        mutation = mutations[i]
        differences = ""
        min_len = min(len(reference_genome), len(mutation))
        
        
        for j in range(min_len):
            if(reference_genome[j] != mutation[j]):
                differences += reference_genome[j]
            else:
                differences += "-"
                
        out_file_2.write(reference_genome)
        out_file_2.write(differences)
    
find_mutations()

#### Mutation Results
Below are the results of searching for the D614G missense mutation amongst the data set.

In [68]:
%%bash
cat ./mutation_data/results.txt

4 - G
21 - G
24 - G


### Pulling Data from NCBI

There are third party resources to pull data from the NCBI database. Utilizing thee resources to streamline or automate collecting sequences would be convinient, but they were either too simple to be useful or too complicated to be worth the time put in. 

Furthermore, there are problems with the NCBI database itself. The data labelled as SARS-CoV-1 and SARS-Cov-2 were part of the same dataset, so manual verification is required to see that you're getting the proper sequences. Thus, any attempt at automation would run into this problem and it is likely there are simliar issues with other data entries.


### Relevant GitHubs

Analysis using k-mer composition
https://anderson-github-classroom.github.io/csc-448-project/eagranof/

Even more analysis using k-mer composition
https://anderson-github-classroom.github.io/csc-448-project/skurdogh/ 

Vitulgin Experimentation
https://anderson-github-classroom.github.io/csc-448-project/cilg/

The Levenshtein distance experiment
https://anderson-github-classroom.github.io/csc-448-project/awengel/ 

Mutation Rate Comparison and Spike Proteins
https://anderson-github-classroom.github.io/csc-448-project/pamidi/

## References

(1) Korber, B., Fischer, W. M., Gnanakaran, S., Yoon, H., Theiler, J., Abfalterer, W., . . . Wyles, M. D. (2020). Tracking Changes in SARS-CoV-2 Spike: Evidence that D614G Increases Infectivity of the COVID-19 Virus. Cell, 182(4). doi:10.1016/j.cell.2020.06.043

(2) Guan, Q., Sadykov, M., Mfarrej, S., Hala, S., Naeem, R., Nugmanova, R., . . . Pain, A. (2020). A genetic barcode of SARS-CoV-2 for monitoring global distribution of different clades during the COVID-19 pandemic. International Journal of Infectious Diseases, 100, 216-223. doi:10.1016/j.ijid.2020.08.052

(3) Chico Harlan, S. (2020, March 20). How coronavirus overwhelmed Italy, with more than 4,000 deaths in one month. Retrieved November 27, 2020, from https://www.washingtonpost.com/world/europe/italy-coronavirus-deaths/2020/03/20/4a385552-6a20-11ea-b199-3a9799c54512_story.html

(4) Isabel, S., Graña-Miraglia, L., Gutierrez, J.M. et al. Evolutionary and structural analyses of SARS-CoV-2 D614G spike protein mutation now documented worldwide. Sci Rep 10, 14031 (2020). https://doi.org/10.1038/s41598-020-70827-z

(5) National Center for Biotechnology Information. (n.d.). Retrieved November 27, 2020, from https://www.ncbi.nlm.nih.gov/

(6) Zhang, L., Jackson, C. B., Mou, H., Ojha, A., Rangarajan, E. S., Izard, T., . . . Choe, H. (2020). The D614G mutation in the SARS-CoV-2 spike protein reduces S1 shedding and increases infectivity. National Institutes of Health. doi:10.1101/2020.06.12.148726

(7) Alex Wigglesworth, R. (2020, January 26). California's first two cases of coronavirus are confirmed in L.A. and Orange counties. Retrieved November 27, 2020, from https://www.sandiegouniontribune.com/news/california/story/2020-01-26/californias-first-two-cases-of-coronavirus-are-confirmed-in-l-a-and-orange-counties

(8) California, S. (n.d.). Tracking COVID-19 in California. Retrieved November 27, 2020, from https://covid19.ca.gov/state-dashboard/ 